## --- Day 9: Smoke Basin ---

These caves seem to be lava tubes. Parts are even still volcanically active; small hydrothermal vents release smoke into the caves that slowly settles like rain.

If you can model how the smoke flows through the caves, you might be able to avoid it and be that much safer. The submarine generates a heightmap of the floor of the nearby caves for you (your puzzle input).

Smoke flows to the lowest point of the area it's in. For example, consider the following heightmap:

    2199943210
    3987894921
    9856789892
    8767896789
    9899965678
    
Each number corresponds to the height of a particular location, where 9 is the highest and 0 is the lowest a location can be.

Your first goal is to find the low points - the locations that are lower than any of its adjacent locations. Most locations have four adjacent locations (up, down, left, and right); locations on the edge or corner of the map have three or two adjacent locations, respectively. (Diagonal locations do not count as adjacent.)

In the above example, there are four low points, all highlighted: two are in the first row (a 1 and a 0), one is in the third row (a 5), and one is in the bottom row (also a 5). All other locations on the heightmap have some lower adjacent location, and so are not low points.

The risk level of a low point is 1 plus its height. In the above example, the risk levels of the low points are 2, 1, 6, and 6. The sum of the risk levels of all low points in the heightmap is therefore 15.

Find all of the low points on your heightmap. What is the sum of the risk levels of all low points on your heightmap?

In [3]:
import numpy as np
import pandas as pd
np.__version__, pd.__version__

('1.20.3', '1.3.4')

In [4]:
with open("day9.txt") as f:
    a2d = [[int(n) for n in line.strip() ] for line in f.readlines()]
a2d[:1]

[[5,
  7,
  9,
  6,
  7,
  9,
  8,
  6,
  2,
  1,
  2,
  3,
  7,
  9,
  9,
  5,
  4,
  9,
  8,
  7,
  6,
  5,
  4,
  3,
  4,
  5,
  6,
  7,
  9,
  8,
  7,
  5,
  4,
  2,
  9,
  9,
  9,
  7,
  6,
  5,
  6,
  7,
  9,
  9,
  8,
  7,
  5,
  4,
  5,
  6,
  7,
  9,
  1,
  0,
  9,
  8,
  7,
  8,
  9,
  9,
  9,
  8,
  7,
  7,
  8,
  9,
  9,
  7,
  8,
  9,
  8,
  7,
  6,
  5,
  3,
  2,
  1,
  2,
  3,
  4,
  5,
  6,
  9,
  9,
  8,
  9,
  9,
  9,
  8,
  7,
  6,
  8,
  8,
  7,
  8,
  9,
  9,
  9,
  2,
  1]]

In [5]:
df = pd.DataFrame(a2d)
df.shape

(100, 100)

In [6]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,5,7,9,6,7,9,8,6,2,1,...,6,8,8,7,8,9,9,9,2,1
1,4,6,4,5,9,7,6,4,3,4,...,5,6,7,6,7,9,8,7,9,9
2,3,2,3,4,9,8,7,5,4,5,...,4,3,4,5,9,8,5,6,7,8
3,4,3,5,6,7,9,8,6,7,9,...,3,2,3,9,8,7,4,5,7,9
4,6,4,6,7,9,9,9,7,8,9,...,1,0,9,8,7,6,3,4,5,6


In [20]:
def isMin(df, x,y, w=100, h=100):
    mins = []
    if x==0 or df.iloc[y,x-1] > df.iloc[y,x]:
        mins.append(True)
    else:
        mins.append(False)
    if x==w-1 or df.iloc[y,x+1] > df.iloc[y,x]:
        mins.append(True)
    else:
        mins.append(False)
    if y==0 or df.iloc[y-1,x] > df.iloc[y,x]:
        mins.append(True)
    else:
        mins.append(False)
    if y==h-1 or df.iloc[y+1,x] > df.iloc[y,x]:
        mins.append(True)
    else:
        mins.append(False)
    return all(mins)

In [23]:
isMin(df, 91, 4)

True

In [24]:
df.iloc[91,4]

5

In [26]:
risks = []
for x in range(100):
    for y in range(100):
        if isMin(df, x, y):
            risks.append(df.iloc[y,x]+1)
sum(risks)

572

In [29]:
seen = df > 10 # easy way to create Booleans
seen.shape

(100, 100)

In [30]:
seen.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [33]:
def getBasin(df, seen, x, y, w=100, h=100):
    if x == -1 or x == w:
        return 0
    if y == -1 or y == h:
        return 0
    if seen.iloc[y,x]:
        return 0
    seen.iloc[y,x] = True
    if df.iloc[y,x] == 9:
        return 0
    return (1 
            + getBasin(df, seen, x+1, y) 
            + getBasin(df,seen,x-1,y)
            + getBasin(df,seen,x,y-1)
            + getBasin(df,seen,x,y+1)
           )

In [35]:
basins = []
visited = seen.copy()
for x in range(100):
    for y in range(100):
        basins.append(getBasin(df, visited, x, y))
sum(basins)

7148

In [37]:
sorted(basins)
basins[:3]

[28, 0, 0]

In [39]:
seen.sum().sum()

0

In [40]:
visited.sum().sum()

10000

In [41]:
top_bas = sorted(basins)
top_bas[:5]

[0, 0, 0, 0, 0]

In [42]:
top_bas[-3:]

[92, 93, 99]

In [43]:
sum(top_bas[-3:])

284

In [44]:
top_bas[-3]*top_bas[-2]*top_bas[-1]

847044